In [1]:
import tensorflow as tf 
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
tf.__version__

'2.2.0'

In [ ]:
directory_name = ''
filename = os.path.join(directory_name, 'ratings.csv')
dataset = pd.read_csv(filename, sep=',')

X_Data = np.asanyarray(dataset.sort_values(['userId', 'timestamp']).groupby('userId')['timestamp'].apply(list))
X_data = dataset.sort_values(['userId', 'timestamp']).groupby('userId')['movieId'].apply(list)
X_data = tf.keras.preprocessing.sequence.pad_sequences(X_data)

l = []
for i in range(1, len(X_data)):
    l1 = []
    for e in X_data[i]:
        l1.append(str(e))
    l.append(l1)

In [3]:
import tensorflow as tf
class CustomLoss(tf.keras.losses.CosineSimilarity):
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def call(self, y_true, y_pred):
        return super().call(model.layers[0](y_true), model.layers[0](y_pred))

class ReverseEmbedding(tf.keras.layers.Layer):
    def __init__(self):
        super(ReverseEmbedding, self).__init__()
        
    def get_config(self):
        return {}
        
    def call(self, inputs):
        dot_product = tf.matmul(
            tf.nn.l2_normalize(model.layers[0].weights[0], axis=1),
            tf.nn.l2_normalize(inputs, axis=1),
            transpose_b=True)
        return tf.argmax(dot_product,axis=0)

vocab_size=100
vector_size=30
max_val = 209171

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_val, vector_size, mask_zero=True))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(vector_size, activation='linear'))
model.add(ReverseEmbedding())
loss = CustomLoss()
model.compile(optimizer='adam', loss=loss)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 30)          6275130   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               81408     
_________________________________________________________________
dense (Dense)                (None, 30)                3870      
_________________________________________________________________
reverse_embedding (ReverseEm (None,)                   0         
Total params: 6,360,408
Trainable params: 6,360,408
Non-trainable params: 0
_________________________________________________________________
None


In [2]:
import numpy as np
# train_x = np.random.randint(1000, size=(100,3014))
# train_y = np.random.randint(1000, size=(100,1))
checkpoint = tf.keras.callbacks.ModelCheckpoint('keras_model.hdf5')
history = model.fit(train_x, train_y, epochs=1, batch_size=32, validation_split=0.2, callbacks=[checkpoint]
                   , workers=2)

3/3 [==============================] - 3s 1s/step - loss: 9.5154e-04 - val_loss: -2.1153e-04


In [11]:
# model.fit(, , epochs=1, batch_size=1)
l = [[132, 213, 4234, 4342, 3223, 32], [32, 421, 21,432]]
for i in l:
    model.fit([i[:-1]], [l[-1]])

1/1 [==============================] - 0s 881us/step - loss: -0.0388
